## シミュレーションを流し、学習するファイル

In [14]:
#path
pdb_filepath = "./alanine-dipeptide-nowater.pdb" #pdbのパス
ff_start_filepath  = "./GB99dms_start.xml" #startのffのパス
out_dir = "./outdir/" #出力するディレクトリ
isdir(out_dir) || mkdir(out_dir)

#setting
username = "ysako" #自分のusername
n_jobs = 5 #並列して流すシミュレーションの数
nsteps = 100000 #シミュレーションのステップ数

maximum_update_ratio = 0.01 #forcefieldをどの程度変えるか

0.01

## 現在のepochの情報

In [5]:
epoch_n = 1 # 現在のエポック
epoch_dir = joinpath(out_dir, "epoch_$(epoch_n)") #epochごとにディレクトリを作る
isdir(epoch_dir) || mkdir(epoch_dir)
ff_dir = joinpath(epoch_dir, "forcefield") #シミュレーションに用いるffのディレクトリのパス
isdir(ff_dir) || mkdir(ff_dir)

"./outdir/epoch_1/forcefield"

## 最初のforcefieldからシミュレーションに用いるxmlファイルを作成

In [6]:
for i in 1:n_jobs
    ff_filepath = ff_start_filepath
    ff_output_xml_filepath = joinpath(ff_dir, "epoch_$(epoch_n)_$(i).xml")
    ff_output_txt_filepath = joinpath(ff_dir, "epoch_$(epoch_n)_$(i).txt")
    if i == 1
        run(`python convert_xml_to_txt.py $(ff_filepath) $(ff_output_txt_filepath)`)
        run(`cp $(ff_filepath) $(ff_output_xml_filepath)`)
    else
        run(`python create_ff.py $(ff_filepath) $(ff_output_xml_filepath) $(ff_output_txt_filepath) $(maximum_update_ratio)`)
    end
end

## targetと現在のパラメータがどの程度ずれているか

In [7]:
# txtファイルから力場パラメータを取り出す関数
function output_ff_parameter(ff_input_txt_filepath)
    f = open(ff_input_txt_filepath, "r")
    datalist = readlines(f)
    close
        
    #output
    theta_zero_phi = []
    k_phi = []
    n_phi = []
    theta_zero_psi = []
    k_psi = []
    n_psi = []
    
    current = "PHI"   
    #create phi parameter
    for line in datalist
        if(line == "PHI_PARAMETER")
            continue
        #"PSI_PARAMETER"が現れたらcurrent->"PSI"
        elseif(line == "PSI_PARAMETER")
            current = "PSI"
            continue
        end        
        #parameterを配列に格納していく
        #phi
        if(current == "PHI")
            if(occursin(r"type.*", split(line)[1]))
                continue
            elseif(occursin(r"periodicity.*", split(line)[1]))
                push!(n_phi, parse(Float64, split(line)[2]))
            elseif(occursin(r"phase.*", split(line)[1]))
                push!(theta_zero_phi, parse(Float64, split(line)[2]))
            elseif(occursin(r"k.*", split(line)[1]))
                push!(k_phi, parse(Float64, split(line)[2]))
            else
                println("ERROR")
            end
        end 
        #psi
        if(current == "PSI")
            if(occursin(r"type.*", split(line)[1]))
                continue
            elseif(occursin(r"periodicity.*", split(line)[1]))
                push!(n_psi, parse(Float64, split(line)[2]))
            elseif(occursin(r"phase.*", split(line)[1]))
                push!(theta_zero_psi, parse(Float64, split(line)[2]))
            elseif(occursin(r"k.*", split(line)[1]))
                push!(k_psi, parse(Float64, split(line)[2]))
            else
                println("ERROR")
            end
        end
    end
    return theta_zero_phi, k_phi, n_phi, theta_zero_psi, k_psi, n_psi
end

output_ff_parameter (generic function with 1 method)

In [20]:
target_ff_filepath = "./target.txt"
_, k_phi_target, _, _, k_psi_target, _ = output_ff_parameter(target_ff_filepath)

current_ff_filepath = joinpath(ff_dir, "epoch_$(epoch_n)_1.txt")
_, k_phi_current, _, _, k_psi_current, _ = output_ff_parameter(current_ff_filepath)

@show phi_dif = sum((k_phi_target .- k_phi_current) .^ 2)
@show psi_dif = sum((k_psi_target .- k_psi_current) .^ 2)

phi_dif = sum((k_phi_target .- k_phi_current) .^ 2) = 0.003292228289939123
psi_dif = sum((k_psi_target .- k_psi_current) .^ 2) = 0.013468668279455153


0.013468668279455153

## シミュレーション

In [11]:
# 一つのシミュレーションを流すジョブスクリプト
# ff_filepath　　　  :xml formatの力場ファイルのパス
# epoch_dir         :アウトプットするディレクトリのパス
# i                 :流すシミュレーションの通し番号

function run_job(ff_filepath, epoch_n, out_traj_dir, i)
    sbatch_file = "temp_epoch$(epoch_n)_$(i).sh"
    traj_filepath = joinpath(out_traj_dir, "traj")
    
    open(sbatch_file, "w") do of
        println(of, "#!/bin/bash")
        println(of, "#SBATCH -p all")
        println(of, "#SBATCH -J sim$(i) # job name")
        println(of, "#SBATCH -n 1  # num of total mpi processes")
        println(of, "#SBATCH -c 1  # num of threads per mpi processes")
        println(of, "#SBATCH -o $(out_traj_dir)/run$(i).log")
        println(of, "#SBATCH -w n4")
        println(of, "python sim.py $(pdb_filepath) $(ff_filepath) $(traj_filepath)$(i) $(nsteps)")
    end
    run(`sbatch $(sbatch_file)`)
    sleep(5)
    rm(sbatch_file)
end

run_job (generic function with 1 method)

In [12]:
#epoch_n
function run_jobs(epoch_n)
    epoch_dir = joinpath(out_dir, "epoch_$(epoch_n)") #epoch_nで出力するディレクトリ
    isdir(epoch_dir) || mkdir(epoch_dir)

    out_traj_dir = joinpath(epoch_dir, "traj")
    isdir(out_traj_dir) || mkdir(out_traj_dir)
    
    completed = false #全てのシミュレーションを終えたらtrueに

    ff_dir = joinpath(epoch_dir, "forcefield") #xml formatのディレクトリ
    #njobsの数だけシミュレーションを並列に流す
    for i in 1:n_jobs
        ff_filepath = joinpath(ff_dir, "epoch_$(epoch_n)_$(i).xml")
        run_job(ff_filepath, epoch_n, out_traj_dir, i)
    end

    #シミュレーションが全て終わるまで待機
    while(!completed)
        n_completed = n_jobs - length(readlines(`squeue -u $(username)`)) + 1 #終了したシミュレーションの数
        #シミュレーションが終わっていないとき
        if(n_completed < n_jobs)
            println("--- $(n_completed) of $(n_jobs) jobs completed")
            sleep(120) #2分待機
        #全てのシミュレーションが終わったとき
        else
            completed = true
        end
    end
    println("--- all jobs completed")
end

run_jobs (generic function with 1 method)

In [ ]:
epoch_n = 1
@time run_jobs(epoch_n)

Submitted batch job 152
Submitted batch job 153
Submitted batch job 154
Submitted batch job 155
Submitted batch job 157
--- 0 of 5 jobs completed
--- 0 of 5 jobs completed
--- 0 of 5 jobs completed
--- 0 of 5 jobs completed
--- 0 of 5 jobs completed
--- 0 of 5 jobs completed
--- 0 of 5 jobs completed
--- 0 of 5 jobs completed
--- 0 of 5 jobs completed
--- 0 of 5 jobs completed
--- 0 of 5 jobs completed
--- 0 of 5 jobs completed


## dcdファイルをtxtファイルに変換

In [22]:
input_dir = joinpath(epoch_dir, "traj")
output_dir = joinpath(epoch_dir, "dihedral")
isdir(output_dir) || mkdir(output_dir)
for i in 1:n_jobs
    dcd_filepath = joinpath(input_dir, "traj$(i).dcd")
    output_filepath = joinpath(output_dir, "traj$(i).txt")
    run(`python create_dihedral_traj.py $(pdb_filepath) $(dcd_filepath) $(output_filepath)`)
end

In [21]:
epoch_dir

"./outdir/epoch_1"